In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import vectorbt as vbt
import yfinance as yf
from talipp.indicators import  RSI
import pandas_ta as ta
import plotly
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def get_stock_data(symbol, start_date, end_date,interval):
    try:
        # Download the stock price data from Yahoo Finance API
        df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)
        df.columns = [x.lower() for x in df.columns]
        
        # Calculate MACD values
        df.ta.macd(close='close', fast=12, slow=26, append=True)
        
        # Force lowercase (optional)
        df.columns = [x.lower() for x in df.columns]
        
        # Calculate emas
        df["ema200"] = df['close'].ewm(span=200, adjust=False, min_periods=12).mean()
        df["ema100"] = df['close'].ewm(span=100, adjust=False, min_periods=12).mean()
        df["ema50"] = df['close'].ewm(span=50, adjust=False, min_periods=12).mean()
        
        # Initialize columns for entries, exits, sell entries, and sell exits
        df['entries'] = False
        df['exits'] = False
        df['sell_entries'] = False
        df['sell_exits'] = False

        return df
    
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return None

In [ ]:
def calculate_portfolio_stats(df, init_cash=10000, fees=0.002):
    entries = df['entries']
    exits = df['exits']
    short_entries = df['sell_entries']
    short_exits = df['sell_exits']
    data = df['close']
    # Create a Portfolio instance from signals
    pf = vbt.Portfolio.from_signals(
        data, entries, exits, short_entries, short_exits,
        init_cash=init_cash, fees=fees
    )
    
    # Return portfolio stats
    return pf.stats(silence_warnings=True)


In [ ]:
# Function to calculate average for numerical values in a row
def calculate_average(row):
    # Convert the values in the row to numeric, ignoring non-numeric values
    numerical_values = pd.to_numeric(row, errors='coerce')
    
    # Check if there are numerical values in the row
    if not numerical_values.empty:
        # Calculate the mean of the numerical values
        return np.mean(numerical_values)
    
    # If no numerical values, return NaN
    return np.nan